In [1]:
import pandas as pd
import numpy as np
import requests


In [2]:
from bs4 import BeautifulSoup
paras = []
for i in range(1, 10):
    url = "https://truecrimedaily.com/search/?q=philadelphia&page=" + str(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    print(soup)
    para = soup.find_all('a', class_ = "gridler__card-link gridler__card-link--default js-track-link js-click-article") 
    paras.append(para)


<!DOCTYPE html>

<html class="device-view-desktop viewer-country-us" data-device-view="desktop" data-env="prod" data-viewer-country="US" lang="en">
<!--
app_version=3.1.3
benchmark=60ms
etag=61104451924169b39159734306678678
timestamp=1669515202
timestamp_iso=2022-11-27T02:13:22.273658Z
screen=desktop-search
-->
<head>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="en" http-equiv="content-language"/>
<meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="unsafe-url" name="referrer"/>
<title>Search - philadelphia - news | Truecrimedaily.com</title>
<meta content="-6s5z6nXpsQ84w7IPVM1rIQ2efKrEBSj1gClF7jaRbs" name="google-site-verification">
<meta content="p8egs8nzm3mepo9ry5spzdakx3wjgy" name="facebook-domain-verification">
<meta content="noindex,follow" name="robots"/>
<meta content="838695666213761" property="fb:app_id"/>
<meta content=

In [3]:
links = []
for i in range(len(paras)):
    for j in range(len(paras[i])):
        links.append(paras[i][j].get('href'))

In [4]:
text = []
for i in range(len(links)):
    url = links[i]
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    para = soup.find_all('p')
    text.append(para)
    

In [5]:
dates = []
titles = []
for i in range(len(links)):
    url = links[i]
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    para = soup.find_all('h5', class_ = "article__published-info")
    dates.append(para)
    para = soup.find_all('h1', class_ = "article__header-title")
    titles.append(para)

In [8]:
for i in range(len(dates)):
    dates[i] = str(dates[i])
    dates[i] = dates[i].replace('[<h5 class="article__published-info">', '')
    dates[i] = dates[i].replace('</h5>]', '')
    titles[i] = str(titles[i])
    titles[i] = titles[i].replace('[<h1 class="article__header-title">', '')
    titles[i] = titles[i].replace('</h1>]', '')        

In [11]:
for i in range(len(dates)):
    dates[i] = dates[i].replace('\n', '')

In [13]:
for i in range(len(dates)):
    dates[i] = dates[i].strip()    

In [17]:
dict = {}
for i in range(len(links)):
    dict[links[i]] =  [titles[i], dates[i], text[i]]

In [22]:
## creating a fucntion to remove html tags from the text data in the dictionary

def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

for i in range(len(links)):
    for j in range(len(dict[links[i]][2])):
        dict[links[i]][2][j] = remove_html_tags(str(dict[links[i]][2][j]))
        dict[links[i]][2][j] = dict[links[i]][2][j].strip()
        dict[links[i]][2][j] = dict[links[i]][2][j].replace('\n', '')
        dict[links[i]][2][j] = dict[links[i]][2][j].replace('\t', '')
        dict[links[i]][2][j] = dict[links[i]][2][j].replace('\r', '')

In [24]:
## converting ictionary into data frames with columns links, titles, dates, text
df = pd.DataFrame.from_dict(dict, orient='index', columns=['titles', 'dates', 'text'])


In [25]:
df.head()

,titles,dates,text
https://truecrimedaily.com/2022/10/28/philadelphia-former-sheriff-samir-ahmad-accused-selling-guns-used-roxborough-scrimmage-shooting/,"Former Philly deputy accused of selling guns, ...",10/28/2022 2:38 pm PDT,[PHILADELPHIA (TCD) -- A 29-year-old man faces...
https://truecrimedaily.com/2022/09/29/3-arrested-rapper-pnb-rock-los-angeles-california-father-son-freddie-lee-trone-shauntel-trone/,Stepmother-father-son trio charged for rap art...,09/29/2022 3:01 pm PDT,[LOS ANGELES (TCD) -- A stepmother-son duo has...
https://truecrimedaily.com/2022/09/28/philadelphia-football-scrimmage-shooting-roxborough-northeast-boys-latin-high-school-student-killed-injured/,"14-year-old student killed, 4 injured in shoot...",09/28/2022 12:48 pm PDT,[PHILADELPHIA (TCD) -- Police are looking for ...
https://truecrimedaily.com/2022/09/13/philadelphia-rapper-pnb-rock-fatally-shot-in-los-angeles-restaurant-california-rakim-hasheem-allen/,Philadelphia rapper PnB Rock fatally shot whil...,09/13/2022 12:50 pm PDT,[LOS ANGELES (TCD) -- Philadelphia rapper PnB ...
https://truecrimedaily.com/2022/09/02/ohio-teen-father-arrested-1-year-old-child-dies-left-in-hot-car-landon-parrot/,Ohio teen father accused of deliberately leavi...,09/02/2022 2:04 pm PDT,"[NEW PHILADELPHIA, Ohio (TCD) -- A 19-year-old..."


In [26]:
df = df.reset_index()
df = df.rename(columns={'index': 'links'})

In [28]:
## removing '[', ']' from the text column
df['text'] = df['text'].astype(str)
df['text'] = df['text'].str.replace('[', '')
df['text'] = df['text'].str.replace(']', '')


/var/folders/fj/r2kb_f4d3k1gxmcwsdc81_1r0000gn/T/ipykernel_46692/1075762652.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['text'] = df['text'].str.replace('[', '')
/var/folders/fj/r2kb_f4d3k1gxmcwsdc81_1r0000gn/T/ipykernel_46692/1075762652.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['text'] = df['text'].str.replace(']', '')


In [31]:

df['date'] = df['dates'].str.split(' ').str[0]
df['time'] = df['dates'].str.split(' ').str[1] + ' ' + df['dates'].str.split(' ').str[2]


df = df.drop(['dates'], axis = 1)

In [32]:
df.head()

,links,titles,text,date,time
0,https://truecrimedaily.com/2022/10/28/philadel...,"Former Philly deputy accused of selling guns, ...",'PHILADELPHIA (TCD) -- A 29-year-old man faces...,10/28/2022,2:38 pm
1,https://truecrimedaily.com/2022/09/29/3-arrest...,Stepmother-father-son trio charged for rap art...,'LOS ANGELES (TCD) -- A stepmother-son duo has...,09/29/2022,3:01 pm
2,https://truecrimedaily.com/2022/09/28/philadel...,"14-year-old student killed, 4 injured in shoot...",'PHILADELPHIA (TCD) -- Police are looking for ...,09/28/2022,12:48 pm
3,https://truecrimedaily.com/2022/09/13/philadel...,Philadelphia rapper PnB Rock fatally shot whil...,'LOS ANGELES (TCD) -- Philadelphia rapper PnB ...,09/13/2022,12:50 pm
4,https://truecrimedaily.com/2022/09/02/ohio-tee...,Ohio teen father accused of deliberately leavi...,"'NEW PHILADELPHIA, Ohio (TCD) -- A 19-year-old...",09/02/2022,2:04 pm


In [33]:
df.shape

(133, 5)

In [34]:
df.to_csv('true_crime_daily.csv', index = True)
